<a href="https://colab.research.google.com/github/priyankavaids/Week-3_EV_Chatbot/blob/main/Copy_of_03_ev_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nltk transformers torch
import nltk
from transformers import pipeline


In [ ]:

ev_faq = {
    "battery": "Battery capacity is measured in kWh. Larger kWh usually gives more range. Typical values: 30–80 kWh.",
    "range": "Real-world EV range varies. Many modern EVs offer 250–500 km per charge depending on driving and conditions.",
    "charging": "Charging time depends on charger power. Fast DC chargers (50-150 kW) can give ~80% in 30-60 minutes. Home AC chargers (3-7 kW) take several hours.",
    "price": "EV price depends on make/model and battery. In India, popular models start from around ₹10-20 Lakh (varies widely).",
    "efficiency": "Efficiency is often measured in Wh/km or km/kWh. Lower Wh/km = better efficiency.",
    "stations": "Use apps like PlugShare or Google Maps to find nearby charging stations."
}

def get_rule_answer(text):
    text = text.lower()
    for k in ev_faq:
        if k in text:
            return ev_faq[k]
    return None

print("EV RuleBot ready. Type 'quit' to stop.")
while True:
    user = input("You: ").strip()
    if user.lower() in ("quit","exit"):
        print("EVBot: Bye! ⚡")
        break
    # try rule-based
    ans = get_rule_answer(user)
    if ans:
        print("EVBot:", ans)
    else:
        print("EVBot: Sorry, I don't know that yet. Ask about battery, range, charging, stations, price, or efficiency.")


EV RuleBot ready. Type 'quit' to stop.


KeyboardInterrupt: Interrupted by user

In [ ]:
# Corrected interactive chatbox (use with the rule-based get_rule_answer function)
from ipywidgets import Text, VBox, Output
from IPython.display import display

out = Output()
text = Text(description="You:")

def on_enter(widget):                # handler receives the Text widget (not a dict)
    user_input = widget.value.strip()  # get the text typed by the user
    with out:
        if not user_input:
            return
        if user_input.lower() in ("quit","exit"):
            print("EVBot: Bye ⚡")
            text.disabled = True
            return
        # get answer from your rule-based FAQ function
        ans = get_rule_answer(user_input)   # you defined this earlier
        print(f"You: {user_input}")
        if ans:
            print(f"EVBot: {ans}\n")
        else:
            print("EVBot: Sorry, I don't know that yet. Ask about battery, range, charging, stations, price, or efficiency.\n")
    widget.value = ""   # clear input box after handling

text.on_submit(on_enter)   # attach handler
display(VBox([text, out]))


In [ ]:
# Fixed styled chat UI for Colab (uses ipywidgets.HTML so VBox accepts it)
from ipywidgets import Text, VBox, Button, HTML, Output
from IPython.display import display, clear_output
import difflib, html

# --- Rich FAQ (if you already have EV_FAQ or get_rule_answer, you can skip redefining)
EV_FAQ = {
    "battery": "Battery capacity is measured in kWh. Larger kWh generally means longer range. Typical passenger EV batteries: 30–80 kWh.",
    "range": "EV range depends on battery size, vehicle weight, driving speed and climate. Many modern EVs achieve 250–500 km per full charge.",
    "charging": "Charging time depends on charger power. Fast DC chargers (50–150 kW) can charge to ~80% in 30–60 mins; home AC chargers (3–7 kW) take several hours.",
    "charging stations": "Use apps like PlugShare, Google Maps or local utilities to find public charging stations near you.",
    "efficiency": "Efficiency is measured in Wh/km or km/kWh. Lower Wh/km is better (less energy needed per km).",
    "maintenance": "EVs have fewer moving parts vs ICE vehicles — less oil changes, but battery health and tire wear matter.",
    "incentives": "Many governments offer incentives/subsidies for EV purchases and tax benefits — check your local/state policies.",
    "cost": "Total cost depends on purchase price, electricity cost, incentives, and maintenance savings vs petrol vehicles.",
    "fast charging": "Fast charging is convenient but frequent use can slightly increase long-term battery degradation. Balance with slower charging when possible.",
    "home_charging": "Install a dedicated home charger (wallbox) with proper earthing. Typical home chargers use 3–7 kW power.",
    "safety": "EV batteries are designed with many safety systems. Follow manufacturer guidelines and use certified chargers.",
    "range_per_kwh": "Range per kWh = (range_km / battery_kwh). Higher value means better efficiency.",
    "regen_braking": "Regenerative braking recovers some energy during deceleration, improving efficiency in city driving.",
    "payload_effect": "Heavier loads reduce range. For best range, carry only necessary weight and maintain proper tire pressure."
}

def get_rule_answer(text):
    text_low = text.lower().strip()
    for k in EV_FAQ:
        if k in text_low:
            return EV_FAQ[k]
    keys = list(EV_FAQ.keys())
    match = difflib.get_close_matches(text_low, keys, n=1, cutoff=0.6)
    if match:
        return EV_FAQ[match[0]]
    for word in text_low.split():
        m = difflib.get_close_matches(word, keys, n=1, cutoff=0.7)
        if m:
            return EV_FAQ[m[0]]
    return None

# --- HTML/CSS for chat bubbles
CHAT_CSS = """
<style>
.ev-chat-container{font-family: Arial; max-width:760px; margin:6px 0;}
.ev-msg{padding:8px 12px; border-radius:14px; margin:6px 0; display:inline-block; max-width:80%;}
.ev-user{background:#e6f3ff; color:#03396c; align-self:flex-end;}
.ev-bot{background:#f1f1f1; color:#111; align-self:flex-start;}
.ev-row{display:flex; flex-direction:row;}
.ev-row.user{justify-content:flex-end;}
.ev-row.bot{justify-content:flex-start;}
.small{font-size:11px; color:#666; margin-top:4px;}
</style>
"""

def render_messages_widget(history):
    html_lines = [CHAT_CSS, '<div class="ev-chat-container">']
    for role, text in history:
        role_class = "user" if role=="You" else "bot"
        bubble_class = "ev-user" if role=="You" else "ev-bot"
        safe_text = html.escape(text).replace("\n","<br>")
        html_lines.append(f'<div class="ev-row {role_class}"><div class="ev-msg {bubble_class}"><b>{role}:</b> {safe_text}</div></div>')
    html_lines.append("</div>")
    # Return an ipywidgets.HTML widget (not IPython.display.HTML)
    return HTML(value="\n".join(html_lines))

# --- Interactive UI ---
history = []  # list of tuples (role, text)
text = Text(description="You:", placeholder="Type your EV question and press Enter")
clear_btn = Button(description="Clear", button_style='warning')
out = Output()

def on_submit(widget):
    user_text = widget.value.strip()
    if not user_text:
        return
    history.append(("You", user_text))
    ans = get_rule_answer(user_text)
    if ans:
        bot_reply = ans
    else:
        bot_reply = "Sorry, I don't know that yet. Try asking about battery, range, charging, stations, incentives, or efficiency."
    history.append(("EVBot", bot_reply))
    widget.value = ""
    # update display inside 'out' Output
    with out:
        clear_output(wait=True)
        display(render_messages_widget(history))

def on_clear(b):
    history.clear()
    with out:
        clear_output(wait=True)
        display(HTML("<i>Chat cleared. Start a new conversation.</i>"))

# hook events
text.on_submit(lambda w: on_submit(w))
clear_btn.on_click(on_clear)

# display UI (VBox accepts ipywidgets: Text, Button, HTML/Output)
display(VBox([text, clear_btn, out]))
# initialize empty view
with out:
    display(render_messages_widget(history))


In [1]:
import os
print(os.getcwd())


/content


In [2]:
import os
os.listdir()


['.config', 'sample_data']

In [5]:
file_path = "03_ev_chatbot (1).ipynb"
clean_file_path = "03_ev_chatbot_clean.ipynb"
